In [9]:
import pyvista as pv 
import pyransac3d as pyrsc

from utils.statistical_tools import normalize_points

In [10]:
POOL_NAME = '3-4'
PROJECT_NAME = 'pozas_clasificada_110125_100_10_120'
PATH_PROJECT_LAS = f'data/{PROJECT_NAME}/{POOL_NAME}_clasificada_col_0_row_0_19s.las'
LIMIT = -1

In [11]:
import laspy as lp
import numpy as np

def read_las_file(las_path: str):
    try:
        return lp.read(las_path)
    except Exception as e:
        print(f"Error reading {las_path}: {e}")
        return None
    
def normalize_colors(colors):
    rgba = colors - colors.min(axis=0)
    rgba = rgba / rgba.max(axis=0)
    return rgba #(rgba * 255).astype(np.uint8)

def get_points_and_colors_from_las(las_path: str, limit: int = -1):
    if not isinstance(las_path, str):
        raise ValueError("las_path must be a string.")
    if not isinstance(limit, int) or limit < -1:
        raise ValueError("limit must be an integer greater than or equal to -1.")

    las = read_las_file(las_path)
    if las is None:
        raise ValueError("No valid LAS file found.")

    points = np.vstack((las.x, las.y, las.z)).transpose()
    colors = np.vstack((las.red, las.green, las.blue)).transpose()

    if limit > 0:
        idxs = np.random.choice(len(points), limit, replace=False)
        points = points[idxs]
        colors = colors[idxs]

    rgba_colors = normalize_colors(colors)
    return points, colors, rgba_colors, np.expand_dims(las.classification, -1)

In [12]:
las_points, las_colors, las_rgba_colors, las_clasification = get_points_and_colors_from_las(PATH_PROJECT_LAS, limit=LIMIT)
las_points_norm, x_min, x_max, y_min, y_max, z_min, z_max = normalize_points(las_points[:,:3])

las_data_points = np.hstack((las_points_norm, las_rgba_colors, las_clasification))
#las_data_points = np.hstack((las_points, las_rgba_colors, las_clasification))

other_points = las_data_points[las_data_points[:,-1] != 65]
water_points = las_data_points[las_data_points[:,-1] == 65]
print(water_points.shape)
plane = pyrsc.Plane()
best_eq, best_inliers = plane.fit(water_points[:,:3], 0.01)
A, B, C, D  = best_eq 
print(f"Ecuación del plano: {A}x + {B}y + {C}z + {D} = 0")

delta = 0.05
X = other_points[:, 0]
Y = other_points[:, 1]
Z = other_points[:, 2]

# Calcular la distancia de cada punto al plano
distances = (A * X + B * Y + C * Z + D) / np.sqrt(A**2 + B**2 + C**2)

# Crear una malla de puntos para el plano
xx, yy = np.meshgrid(np.linspace(X.min(), X.max(), 10), np.linspace(Y.min(), Y.max(), 10))
zz = (-A * xx - B * yy - D) / C

# # Crear una superficie para el plano
plane = pv.StructuredGrid(xx, yy, zz)

# # Crear un plotter de PyVista
plotter = pv.Plotter()

# Separar los puntos por encima y por debajo del plano
below_plane = distances < -delta
above_plane = distances >= delta

water_points_pv = pv.PolyData(water_points[:,:3])
plotter.add_points(water_points_pv, color='yellow', point_size=5)

# # Añadir los puntos por debajo del plano en verde
points_below = pv.PolyData(np.column_stack((X[below_plane], Y[below_plane], Z[below_plane])))
plotter.add_points(points_below, color='green', point_size=5)

# # Añadir los puntos por encima del plano en rojo
points_above = pv.PolyData(np.column_stack((X[above_plane], Y[above_plane], Z[above_plane])))
plotter.add_points(points_above, color='red', point_size=5)

# # Añadir la superficie del plano al plotter
plotter.add_mesh(plane, color='blue', opacity=0.5)

# # Mostrar el plotter
plotter.show()

# water_points_z = water_points[:,2]
# gmm = GaussianMixture(n_components=1)
# gmm.fit(water_points_z.reshape(-1, 1))
# plot_gmm(water_points_z, gmm)

# water_points_z_mean = np.max(gmm.means_)
# print(water_points_z_mean)

# noise_points = other_points[other_points[:,2] > water_points_z_mean]
# under_water_points = other_points[other_points[:,2] < water_points_z_mean]
 
# plot_pv([water_points[:,:3], noise_points[:,:3], under_water_points[:,:3]],
#         ['blue', 'red', 'green'])

(3686228, 7)
Ecuación del plano: 0.0015600772290981521x + 0.0025687419502828696y + 0.9999954838517183z + -0.12196155490630915 = 0


Widget(value='<iframe src="http://localhost:40815/index.html?ui=P_0x7bad35882f20_0&reconnect=auto" class="pyvi…

In [19]:
import pandas as pd

R = other_points[:, 3]
G = other_points[:, 4]
B = other_points[:, 5]

all_points = np.column_stack((X[below_plane], Y[below_plane], Z[below_plane], R[below_plane], G[below_plane], B[below_plane]))


df = pd.DataFrame(all_points, columns=['x', 'y', 'z', 'r', 'g', 'b'])
df.to_csv(f'3-4.csv', index=False)


In [20]:
np.mean(Z[below_plane] * (z_max - z_min))

np.float64(0.9450095259212878)

In [21]:
print(x_max - x_min)

250.95924999995623


In [22]:
print(y_max -  y_min)

280.1910000005737


In [23]:
z_max - z_min

np.float64(14.376749999999902)

In [24]:
D

np.float64(-0.12196155490630915)